In [43]:
import statsmodels.formula.api as smf
from patsy.contrasts import Diff
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd 
import numpy as np
import seaborn as sns

### Regression 1: 
regresses initial movement vector (initial_theta) on midpoint error (midpoint_error), trial (log_trial) and midpoint sensory uncertainty on the previous trial (previous_uncertainty) as well as the interaction between midpoint error and uncertainty on the previous trial. 

In [36]:
data = pd.read_csv('../data/regression_data.csv')
data

,trial,initial_theta,midpoint_error,previous_uncertainty
0,0,4.012769,2.990414,no_uncertainty
1,1,3.818803,2.163832,no_uncertainty
2,2,3.526050,1.961887,no_uncertainty
3,3,3.359577,1.994340,no_uncertainty
4,4,3.793889,2.467037,no_uncertainty
...,...,...,...,...
295,295,4.817503,2.988111,unlimited_uncertainty
296,296,5.594581,4.154392,unlimited_uncertainty
297,297,4.258937,4.030977,unlimited_uncertainty
298,298,4.967267,4.509882,unlimited_uncertainty


In [42]:
# log transform trials 
data['log_trial'] = data.trial + 1 
data['log_trial'] = np.log10(data.log_trial)
data

,trial,initial_theta,midpoint_error,previous_uncertainty,log_trial
0,0,4.012769,2.990414,no_uncertainty,0.000000
1,1,3.818803,2.163832,no_uncertainty,0.301030
2,2,3.526050,1.961887,no_uncertainty,0.477121
3,3,3.359577,1.994340,no_uncertainty,0.602060
4,4,3.793889,2.467037,no_uncertainty,0.698970
...,...,...,...,...,...
295,295,4.817503,2.988111,unlimited_uncertainty,2.471292
296,296,5.594581,4.154392,unlimited_uncertainty,2.472756
297,297,4.258937,4.030977,unlimited_uncertainty,2.474216
298,298,4.967267,4.509882,unlimited_uncertainty,2.475671


In [54]:
# calculate the change in IMV/initial theta from trial to the next trial 
data['change_initial_theta'] = np.insert(data.initial_theta.values[1:] - data.initial_theta.values[:-1], 0, 0)
data

,trial,initial_theta,midpoint_error,previous_uncertainty,log_trial,change_initial_theta
0,0,4.012769,2.990414,no_uncertainty,0.000000,0.000000
1,1,3.818803,2.163832,no_uncertainty,0.301030,-0.193966
2,2,3.526050,1.961887,no_uncertainty,0.477121,-0.292754
3,3,3.359577,1.994340,no_uncertainty,0.602060,-0.166473
4,4,3.793889,2.467037,no_uncertainty,0.698970,0.434312
...,...,...,...,...,...,...
295,295,4.817503,2.988111,unlimited_uncertainty,2.471292,0.226918
296,296,5.594581,4.154392,unlimited_uncertainty,2.472756,0.777078
297,297,4.258937,4.030977,unlimited_uncertainty,2.474216,-1.335644
298,298,4.967267,4.509882,unlimited_uncertainty,2.475671,0.708330


In [55]:
# create formulas for regressions 
formula = 'initial_theta ~ log_trial + C(previous_uncertainty, Diff) * midpoint_error + 1'
formula_2 = 'change_initial_theta ~ C(previous_uncertainty, Diff) * midpoint_error + 1 '

In [51]:
# Regression 1 
# Create and train the linear regression model with statsmodels
model = smf.ols(formula, data=data)

# fit the model 
reg = model.fit()

# Get the summary of the regression model, including p-values
summary = reg.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:          initial_theta   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     18.12
Date:                Wed, 06 Sep 2023   Prob (F-statistic):           5.88e-22
Time:                        20:23:37   Log-Likelihood:                -523.02
No. Observations:                 300   AIC:                             1064.
Df Residuals:                     291   BIC:                             1097.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------


### Regression 2 
Regresses **change** in IMV (between trials) on midpoint error and midpoint sensory uncertainty on the previous trial as well as the interaction between midpoint error and uncertainty on the previous trial. 

In [56]:
# Regression 2
# Create and train the linear regression model with statsmodels
model_2 = smf.ols(formula_2, data=data)

# fit the model 
reg_2 = model_2.fit()

# Get the summary of the regression model, including p-values
summary_2 = reg_2.summary()
print(summary_2)

                             OLS Regression Results                             
Dep. Variable:     change_initial_theta   R-squared:                       0.058
Model:                              OLS   Adj. R-squared:                  0.036
Method:                   Least Squares   F-statistic:                     2.572
Date:                  Wed, 06 Sep 2023   Prob (F-statistic):             0.0138
Time:                          22:36:02   Log-Likelihood:                -627.33
No. Observations:                   300   AIC:                             1271.
Df Residuals:                       292   BIC:                             1300.
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

### Repeated Measures / Paired T-test 
Compares the last 3 trials of the adaptaiton phase (trial 197 to 199) and the first 3 trials of the washout phase (trial 200 to 202). 

In [61]:
# ttest 
adaptation = data.loc[(data.trial <= 199) & (data.trial >= 197)]
washout = data.loc[(data.trial >= 200) & (data.trial <= 202)]

,trial,initial_theta,midpoint_error,previous_uncertainty,log_trial,change_initial_theta
200,200,5.286212,6.104898,no_uncertainty,2.303196,0.152862
201,201,4.329746,4.706792,unlimited_uncertainty,2.305351,-0.956465
202,202,3.752025,4.115839,unlimited_uncertainty,2.307496,-0.577722


In [63]:
stats.ttest_rel(adaptation.initial_theta, washout.initial_theta)

TtestResult(statistic=3.451073227461378, pvalue=0.07467974164636347, df=2)